<a href="https://colab.research.google.com/github/ArtemDorofeev/IT-education-project/blob/main/%D0%9F%D0%B0%D1%80%D1%81%D0%B5%D1%80_checkroi_BS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install lxml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from bs4 import BeautifulSoup
import lxml

#URL = "https://checkroi.ru/napravlenie/analitika/page/{}"
# или
URL = "https://checkroi.ru/napravlenie/programmirovanie/page/{}"

# Убрать '#' перед адресом, который нужно парсить

In [ ]:
# Формируем суп-объект

def get_soup(page):
  session = requests.session()
  session.headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
        "Accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
  }

  res = session.get(URL.format(page))
  html = res.text
  soup = BeautifulSoup(html, "lxml")
  return soup


In [ ]:
# Собираем все характеристики курса

def get_cource_info(cart):

  # Выбираем название курса
  name_obj = cart.select_one('a.title')
  if name_obj.get_text():
    name = name_obj.get_text()
  else:
    name ='None'

  # Выбираем категорию специальности
  category_obj = cart.select_one('div.pa_kategoriya')
  if category_obj.get_text():
    category = category_obj.get_text()
  else:
    category ='None'

  # Выбираем продолжительность курса
  lng_obj = cart.select_one('div.long')
  if lng_obj.get_text():
    lng = lng_obj.get_text()
  else:
    lng ='None'

  # Выбираем стоимость курса
  price_obj = cart.select_one('div.price')
  if price_obj.get_text():
    price = price_obj.get_text()
  else:
    price ='None'
  
  # Выбираем рейтинг курса
  cource_rate_rev_obj = cart.select_one('div.rating')
  if cource_rate_rev_obj.get_text():
    cource_rate_rev = cource_rate_rev_obj.get_text()
  else:
    cource_rate_rev ='None'
  
  # Выбираем рейтинг школы
  rate_obj = cart.select_one('div.rating-count')
  if rate_obj.get_text():
    rate = rate_obj.get_text()
  else:
    rate ='None'

  # Выбираем количество отзывов школы
  review_obj = cart.select_one('a.comments')
  if review_obj.get_text():
    review = review_obj.get_text()
  else:
    review ='None'

  # Выбираем выдаваемый документ
  doc_obj = cart.select_one('div.vydaetsya-diplom-sertifikat')
  if doc_obj.get_text():
    doc = doc_obj.get_text()
  else:
    doc ='her'

  # Выбираем формат проведения занятий
  format_obj = cart.select_one('div.format-obucheniya')
  if format_obj.get_text():
    format = format_obj.get_text()
  else:
    format ='None'

  # Вытаскиваем внутренний суп из карточки
  link = name_obj.get('href')

  session = requests.session()
  session.headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
        "Accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
  }

  res_in = session.get(link)
  html_in = res_in.text
  soup_in = BeautifulSoup(html_in, "lxml")

  # Ключевые навыки, опыт и стек технологий
  level = soup_in.select("tr.woocommerce-product-attributes-item.woocommerce-product-attributes-item--attribute_pa_uroven td p a")
  key_skills = soup_in.select("tr.attribute_row.attribute_row_chemu-nauchites-na-kurse tr.woocommerce-product-attributes-item.woocommerce-product-attributes-item--attribute_pa_chemu-nauchites td p a")
  key_stecks = soup_in.select("tr.woocommerce-product-attributes-item.woocommerce-product-attributes-item--attribute_pa_tehnologii td p a")
  
  result_dic = {
      'name': name,
      'category': category,
      'long': lng,
      'level': level,
      'key_skills': key_skills,
      'key_stecks': key_stecks,
      'price': price,
      'cource_rate_rev': cource_rate_rev,
      'review': review,
      'rate': rate,
      'diploma': doc,
      'format': format
  }
  return result_dic



In [ ]:
# Собираем список курсов на странице

def get_books_info(soup):
  list_carts = soup.select_one("div.columns-3.products.col_wrap_three.rh-flex-eq-height.woogridrev")
  cart_all = list_carts.select("div.item_product__card")
  result = []
  # Проходимся по каждой карточке и достаем из нее информацию
  for x in cart_all:
    result.append(get_cource_info(x))
  return result



In [ ]:
import time


# 1. Цикл по страницам
# 2. Получить исходный код
# 3. Сварить суп из исходного кода
# 4. Получить информацию о книгах

result = []
# # Цикл по количеству страниц (надо смотреть на сайте сколько!!!)
for page in range(1, 19):
#   Перерыв между запросами
  time.sleep(3)
  soup = get_soup(page)
# Получаем информацию о книгах
  course_info = get_books_info(soup)
# Записываем в result получившуюся инфу о книгах на страниц
  result = result + course_info





In [ ]:
# записываем результат парсинга в csv файлы

import csv

with open("courses_prog_full.csv", "w") as f:
  writer = csv.DictWriter(f, result[0].keys())
  writer.writeheader()
  for r in result:
    writer.writerow(r)

In [ ]:
# Объеденяем полученные csv

import pandas as pd
import glob

In [ ]:
files = glob.glob('*.csv')

In [ ]:
# Объединяем
courses_full = pd.DataFrame()

for file in files:
    data = pd.read_csv(file)
    data['filename'] = file
    courses_full = pd.concat([courses_full, data])

In [ ]:
courses_full.info()

In [ ]:
# Записываем объедененные csv в итоговый Excel

writer = pd.ExcelWriter('dataset_courses.xlsx') # write dataframe to excel 
courses_full.to_excel(writer) # save the excel 
writer.save()